# One dimensional search methods

## Golden Section Search (0th Order)

Assumptions: 
- Unimodal (unique global minimum $x*$ in a given range $[a_0, b_0]$)
- If $a_1 < a_2 < x*$ then $f(x*) < f(a_2) < f(a_1)$
- If $x* < a_1 < a_2$ then $f(x*) < f(a_1) < f(a_2)$

Algorithm:
  Starting from 

In [24]:
import math
from functools import lru_cache

@lru_cache(maxsize=2)
def f(x):
    """
        Define f(x) here
    """
    return x**2 + 4 * math.cos(x)

def golden_ratio():
    # Used to symmetrically reduce term and make sure only one function evaluation is done per iteration
    return (3 - math.sqrt(5)) / 2

def golden_section_search(f, a_0, b_0, target_interval=0.1, reduction=golden_ratio()):
    """
        f - function to optimise
        a_0 - starting a which is < b_0
        b_0 - upper bound on starting interval
        target_interval - target difference between b_i - a_i
    """
    # calculate iterations required to solve
    print(f'Reduction factor = {reduction:.3f}, starting interval = {[a_0, b_0]}')
    intervals = math.ceil(math.log(target_interval / 1, (1 - reduction)))
    print(f'N = log_[base=1-reduction](target_interval / 2) = log_[base={1- reduction:.3f}]({target_interval/2:.3f}) = {intervals}')
    print()
    a_i_m1, b_i_m1 = a_0, b_0
    update_a = True 
    for i in range(intervals):
        old_interval = [a_i_m1, b_i_m1]
        # Calculate intermediate points
        a_i = a_i_m1 + reduction * (b_i_m1 - a_i_m1)
        b_i = a_i_m1 + (1 - reduction) * (b_i_m1 - a_i_m1)

        # Evaluate at intermediate points (assume if using golden ratio that one function evaluation is done)
        f_a_i, f_b_i = f(a_i), f(b_i)
        if f_a_i < f_b_i:
            # update_a = False
            # Update interval
            b_i_m1 = b_i
        else:
            # update_a = True
            a_i_m1 = a_i

        print(f'Iteration {i}: a_{i + 1}={a_i:.3f}, b_{i + 1}={b_i:.3f}, f(a_{i+1})={f_a_i:.3f}, f(b_{i+1})={f_b_i:.3f}, old_interval={["%.3f" % x for x in old_interval]}, new_interval={["%.3f" % x for x in [a_i_m1, b_i_m1]]}, interval_size={b_i_m1 - a_i_m1:.3f}')

    print(f'The value that minimises f is located in the interval {["%.3f" % x for x in [a_i_m1, b_i_m1] ]}')


golden_section_search(f, a_0=1, b_0=2, target_interval=0.2)

Reduction factor = 0.382, starting interval = [1, 2]
N = log_[base=1-reduction](target_interval / 2) = log_[base=0.618](0.100) = 4

Iteration 0: a_1=1.382, b_1=1.618, f(a_1)=2.661, f(b_1)=2.429, old_interval=['1.000', '2.000'], new_interval=['1.382', '2.000'], interval_size=0.618
Iteration 1: a_2=1.618, b_2=1.764, f(a_2)=2.429, f(b_2)=2.344, old_interval=['1.382', '2.000'], new_interval=['1.618', '2.000'], interval_size=0.382
Iteration 2: a_3=1.764, b_3=1.854, f(a_3)=2.344, f(b_3)=2.320, old_interval=['1.618', '2.000'], new_interval=['1.764', '2.000'], interval_size=0.236
Iteration 3: a_4=1.854, b_4=1.910, f(a_4)=2.320, f(b_4)=2.317, old_interval=['1.764', '2.000'], new_interval=['1.854', '2.000'], interval_size=0.146
The value that minimises f is located in the interval ['1.854', '2.000']


## Netwon's Method (2nd Order)

1. Minimising a general non-linear function is difficult

    $min f(x)$

2. Basic idea: minimise a quadratic approximation
    
    $min q(x)$
    
    where $q(x) = f(x_k) + f'(x_k)(x - x_k) + \frac{1}{2}f''(x_k)(x - x_k)^2$

3. Minimise quadratic approximation (not necessarily will lead to a minimum)
    
    $0 = q'(x) = f'(x_k) + f''(x_k)(x - x_k)$

4. Use approximate minimiser as new starting point

    $x_{k+1} = x_k - \frac{f'(x_k)}{f''(x_k)}$

In [8]:
# Increasing powers of x
import math 

def f(x):
    return x**2 + 4 * math.cos(x)
    
def f_(x):
    return 2*x - 4 * math.sin(x)

def f__(x):
    return 2 - 4 * math.cos(x)

def newtons_method(x_0, iterations = 10000000, change_threshold=1e-5):
    x_minus_one = x_0
    x_k = x_minus_one + 2
    k = 0
    change = 2*change_threshold
    while change > change_threshold and k < iterations:
        k += 1
        x_k = x_minus_one - f_(x_minus_one) / f__(x_minus_one)
        change = abs(x_minus_one - x_k)
        x_minus_one = x_k
        print(f'Iteration {k}: x_{k} = {x_k:.3f}')

    
newtons_method(5)

Iteration 1: x_1 = -10.989
Iteration 2: x_2 = 1.820
Iteration 3: x_3 = 1.899
Iteration 4: x_4 = 1.896
Iteration 5: x_5 = 1.895


## Secant Method (Quasi-Newton Method) 2nd Order

Second derivative can be hard to find in practice so we can approximate it with

$ f''(x_k) = \frac{f'(x_k) - f'(x_{k - 1})}{x_k - x_{k - 1}}$

So the update becomes

$x_{k + 1} = x_k - f'(x_k) \frac{x_k - x_{k - 1} }{f'(x_k) - f'(x_{k - 1})}$

In [37]:
# Increasing powers of x

def f(x):
    return x**4 - 14 * x**3 + 60 * x **2 - 70 * x
    
def f_(x):
    return 4 * x**3 - 42 * x**2 + 120 * x - 70

def f__(x):
    return 12 * x**2 - 84 * x + 120

def newtons_method(x__1, x_0, iterations=100000, change_threshold=1e-5):
    x_minus_one = x_0
    x_minus_two = x__1
    x_k = x_minus_one + 2
    k = 0
    change = 2*change_threshold
    while change > change_threshold and k < iterations:
        k += 1
        x_k = x_minus_one - f_(x_minus_one) * (x_minus_one - x_minus_two) / (f_(x_minus_one) - f_(x_minus_two))
        change = abs(x_minus_one - x_k)
        x_minus_two = x_minus_one
        x_minus_one = x_k
        print(f'Iteration {k}: x_{k} = {x_k:.3f}')

    
newtons_method(-0.5, -0.6, iterations=10)

Iteration 1: x_1 = 0.330
Iteration 2: x_2 = 0.593
Iteration 3: x_3 = 0.745
Iteration 4: x_4 = 0.778
Iteration 5: x_5 = 0.781
Iteration 6: x_6 = 0.781
Iteration 7: x_7 = 0.781


## Descent Method (1st Order)

1. Given a point $x_k$
2. Derive descent direction $d_k \in R^n$ 

    i.e. $\nabla f(x_k)^T d^k < 0$

    Steepest Descent: Compute gradient at $x_k$
    
    $d_k = - \nabla f(x_k) $

3. Decide on a step-size $\alpha _k$

    Exact search: $ \alpha _k \in argmin_{\alpha >= 0} f(x_k + \alpha d_k)$

    Backtracking and other methods can also be used

4. Transition to the next point

    $x_{k+1} = x_k + \alpha _k d_k$

# Towards a General Newton-Raphson Method

Issues so far with 1-D:
- Only applicable to single dimension
- Algorithm may cycle
- It may fail to find a descent direction as we are not checking if min or max approx 
- It may converge to a saddle point or a local maximum

## Multivariate Newton-Raphson (2nd order)

1. As in 1-D case we construct a **quadratic** approximation around the current iterated $x_k$

$f(x) \approx f(x_k) + \nabla f(x_k)^T(x - x_k) + \frac{1}{2}(x - x_k)^T \nabla ^2 f(x_k)(x - x_k) \triangleq q(x)$

2. Apply FONC to $q(x)$

$ 0 = \nabla q(x) = \nabla f(x_k) + \nabla ^2 f(x_k) (x - x_k)$

3. Assume that $\nabla ^2 f(x_k) \succsim 0$ (i.e. positive definite), then

$ x_{k + 1} = x_k - \nabla ^2 f(x_k)^{-1} \nabla f(x_k) $

In [9]:
import autograd.numpy as np
import autograd
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

def newton(f, x0, tol=0.0001, maxiter=10000):
    g = autograd.jacobian(f)
    h = autograd.hessian(f)

    print(g)
    x = x0
    for k in range(1, maxiter + 1):

        gx = np.squeeze(g(x))
        hx = np.squeeze(h(x))

        delta = np.linalg.multi_dot([np.linalg.inv(hx), gx])

        x = x - delta
        print(f'Iteration {k}: x_{k} = {x}')
        if np.linalg.norm(delta) < tol:
            break

    return x



def f(x):
    return (x[0] + 10 * x[1]) ** 2 + 5 * (x[2] - x[3]) ** 2 + (x[1] - 2 * x[2]) ** 4 + 10 * (x[0] - x[3]) ** 4

x0 = np.array([3., -1., 0., 1.])
print(f(x0))
print(newton(f, x0, maxiter=3))


215.0
<function unary_to_nary.<locals>.nary_operator.<locals>.nary_f at 0x000002615C7C0160>
Iteration 1: x_1 = [1.587 -0.159 0.254 0.254]
Iteration 2: x_2 = [1.058 -0.106 0.169 0.169]
Iteration 3: x_3 = [0.705 -0.071 0.113 0.113]
[0.705 -0.071 0.113 0.113]
